### Import libraries

In [1]:
from scipy.io import mmread
import pandas as pd
import numpy as np
import gzip
from joblib import dump, load
from utils.slice_data_HVG import slice_data_HVG
from utils.data_pretreatment import preprocess_sparse_matrix
from utils.sample_row_ind import sampled_ind_matrix

ModuleNotFoundError: No module named 'utils'

### Load raw gene expression data, transpose and row-compress. Load metadata.

In [2]:
%%time
#data_path = '/home/luana/workspace/data/matrix.mtx'
data_path = 'C:/Users/luana/Documents/thesis/data/matrix.mtx'
data_sparse = mmread(data_path).transpose().tocsr()
metadata = pd.read_csv("../inst/data/2097-Lungcancer_metadata.csv.gz")

print('Shape of the sparse data matrix is', data_sparse.shape)
print(metadata)
print('Unique Cell Types in metadata:', pd.unique(metadata['CellType']))

KeyboardInterrupt: 

In [13]:
print(metadata.columns)
print(pd.unique(metadata['CellFromTumor']))
print(pd.unique(metadata['PatientNumber']))
#print(metadata.loc[metadata['CellFromTumor'] == False, 'TumorType'])
print(pd.unique(metadata['TumorType']))
print(pd.unique(metadata['TumorSite']))
print(pd.unique(metadata['CellType']))

Index(['Cell', 'nGene', 'nUMI', 'CellFromTumor', 'PatientNumber', 'TumorType',
       'TumorSite', 'CellType'],
      dtype='object')
[ True False]
[1 2 3 4 5 6 7 8]
['Lung']
['I' 'M' 'O' 'N' 'U']
['Cancer' 'Myeloid' 'T_cell' 'Mast_cell' 'B_cell' 'Fibroblast' 'Alveolar'
 'EC' 'Erythroblast' 'Epithelial']


### Downsample gene expression data by rows 

In [4]:
# Generate row indices
ind_rows_downsample = sampled_ind_matrix(metadata = metadata, nbr_samples=40000)
print(ind_rows_downsample[:10])

[944, 1018, 1407, 131, 1019, 202, 854, 691, 940, 973]


In [5]:
# Downsampling raw count data, and metadata with generated row indicies
downsampled_sparse = data_sparse[ind_rows_downsample]
metadata_sampled = metadata.iloc[ind_rows_downsample, ]
print('Shape of Downsampled gene expression data: ', downsampled_sparse.shape)
print('Shape of Metadata: ', metadata_sampled.shape)

Shape of Downsampled gene expression data:  (39970, 33694)
Shape of Metadata:  (39970, 8)


In [15]:
# save results
dump(downsampled_sparse, 'data/downsampled_40000_sparse_gzip', compress=('gzip', 3))
metadata_sampled.to_csv('data/metadata_sampled.csv', index=False)

In [7]:
# Load back
#downsampled_sparse_loaded = load('data/downsampled_40000_sparse_gzip')
#print(downsampled_sparse_loaded.shape)

### Quality Control - cells

In [ ]:
import pandas as pd

# Assuming your dataset is a Pandas DataFrame where rows are genes and columns are cells
# The dataset should also have metadata such as mitochondrial gene markers and gene counts

def quality_control(df, mitochondrial_genes, umi_threshold=1000, gene_threshold=500, mito_fraction_threshold=0.2):
    """
    Perform quality control on single-cell RNA-seq data.
    
    Parameters:
    df (pd.DataFrame): Single-cell RNA-seq data, where rows are genes and columns are cells
    mitochondrial_genes (list): List of mitochondrial genes
    umi_threshold (int): Minimum number of UMIs per cell to pass the filter
    gene_threshold (int): Minimum number of genes detected per cell to pass the filter
    mito_fraction_threshold (float): Maximum fraction of mitochondrial genes to pass the filter (e.g., 0.2 for 20%)
    
    Returns:
    pd.DataFrame: Filtered data after QC
    """
    
    # Calculate the total UMIs per cell (sum of counts per column)
    total_umis = df.sum(axis=0)
    
    # Calculate the number of genes detected per cell (non-zero counts per column)
    genes_detected = (df > 0).sum(axis=0)
    
    # Calculate the mitochondrial gene fraction per cell
    mito_counts = df.loc[mitochondial_genes].sum(axis=0)  # sum mitochondrial gene counts per cell
    mito_fraction = mito_counts / total_umis
    
    # Apply filters
    valid_cells = (total_umis > umi_threshold) & \
                  (genes_detected > gene_threshold) & \
                  (mito_fraction < mito_fraction_threshold)
    
    # Filter the dataset based on the valid cells
    filtered_df = df.loc[:, valid_cells]
    
    return filtered_df

# Example of mitochondrial gene list (you'll need to use the actual mitochondrial genes from your dataset)
mitochondrial_genes = ['MT-CO1', 'MT-CO2', 'MT-ND1', 'MT-ND2']  # Replace with actual mitochondrial genes

# Assuming 'df' is your single-cell RNA-seq data (genes x cells)
filtered_data = quality_control(df, mitochondrial_genes)

print(f"Number of cells before QC: {df.shape[1]}")
print(f"Number of cells after QC: {filtered_data.shape[1]}")


### Slice sparse data by top HVG

In [8]:
# Top 10%
data_sp_csr_HVG = slice_data_HVG(downsampled_sparse, perc_top_genes=0.1)
print(data_sp_csr_HVG.shape)
print(data_sp_csr_HVG.__class__)

(39970, 3369)
<class 'scipy.sparse._csr.csr_matrix'>


### Data pre-treatment

In [9]:
%%time
# Preprocess downsampled data, sliced further by top 10 HVG
data_preprocessed_4000_10HVG = preprocess_sparse_matrix(data_sp_csr_HVG)
print(data_preprocessed_4000_10HVG.shape)

(39970, 3369)
CPU times: user 2.13 s, sys: 896 ms, total: 3.03 s
Wall time: 3.04 s


In [10]:
#%%time
# Preprocess downsampled data, all genes
#data_preprocessed_4000 = preprocess_sparse_matrix(downsampled_sparse)
#print(data_preprocessed_4000.shape)

In [13]:
# Save results
f = gzip.GzipFile("data/data_preprocessed_4000_10HVG", "w")
np.save(file=f, arr = data_preprocessed_4000_10HVG)
f.close()

In [12]:
# Load back
#f = gzip.GzipFile('data/data_preprocessed_4000_10HVG, "r"); a = np.load(f)